In [1]:
import numpy as np
from src.util.evaluation import ForecastErrorEvaluation
from src.util.goal import ForecastingGoal, OneStepForecastingGoal
from src.pipeline import IPipeline, Use_ARLinearRegression, Use_ARSupportVectorRegressionLinear, Use_ARSupportVectorRegressionRBF, Use_ARConvolutionalNeuralNetwork, Use_ARLongShortTermMemoryNeuralNetwork, Use_ARMultiLayerPerceptronNeuralNetwork
from test.util import TestSuite


In [2]:
test_suite = TestSuite()
series1 = test_suite.__get_numpy_test_series_from_index__(6)
goal: ForecastingGoal = OneStepForecastingGoal(offset=1)

In [3]:
from src.performer.supervised_learning_performer import SlidingWindowPerformer


SWP1 = SlidingWindowPerformer(feature_length=5, target_length=1, target_offset=1)
SWP2 = SlidingWindowPerformer(feature_length=5, target_length=1, target_offset=2)
SWP3 = SlidingWindowPerformer(feature_length=5, target_length=1, target_offset=3)
SWP4 = SlidingWindowPerformer(feature_length=5, target_length=1, target_offset=4)
SWP5 = SlidingWindowPerformer(feature_length=5, target_length=1, target_offset=5)

In [4]:
testSeries = np.arange(0, 10, 1)
print(testSeries.shape)

(10,)


In [5]:
class MultiStepWorkflow():
    def __init__(self, series, goal: ForecastingGoal):
        self.workflow_stack = []
        view, offset = goal.options()
        self._view = view
        self._offset = offset
        self._goal = goal
        self._pipelines = np.array([])
        
        size = view+offset+5 #5 is ar variable
        self._series = series[:-size]
        self._test_series = series[-size:]

    def __add_to_pipeline__(self, *pipelines: IPipeline):
        self._pipelines = np.array(pipelines)
    
    def __best_model__(self):
        temp = np.array(self._temp_results)
        loss_errors = temp[::, 2]
        min = np.min(loss_errors)
        index = np.where(loss_errors == min)[0][0]
        return temp[index]

    def __execute__(self):
        for i in range(self._view):
            oneStep = OneStepForecastingGoal(offset = i+1)
            self._temp_results = []
            print("Processing Step ",i)
            for i in range(len(self._pipelines)):
                t_result = self._pipelines[i].__execute__(series=self._series, goal=oneStep)
                self._temp_results.append(np.array([t_result[0], t_result[1], t_result[2]]))
                print("Best Models:\t", t_result[0].__identify__(), "\t\t\tError:", str(t_result[2]))

            r = self.__best_model__()
            print("Choosing:"+r[0].__identify__())
            print("--------------------------------------")
            self.workflow_stack.append(r)

    def get(self):
        return self.workflow_stack, self._test_series

In [ ]:
from src.util.goal import MultiStepForecastingGoal


test_suite = TestSuite()
series1 = test_suite.__get_numpy_test_series_from_index__(6)
goal: ForecastingGoal = MultiStepForecastingGoal(offset=1, view=5)
wf = MultiStepWorkflow(series = series1, goal = goal)
wf.__add_to_pipeline__(
    Use_ARSupportVectorRegressionRBF(),
    Use_ARConvolutionalNeuralNetwork(),
    Use_ARSupportVectorRegressionLinear(),
    Use_ARLongShortTermMemoryNeuralNetwork(),
    Use_ARMultiLayerPerceptronNeuralNetwork(),
    Use_ARLinearRegression()
)
wf.__execute__()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(test)

In [9]:
workflow_stack, series_test = wf.get()

In [ ]:
workflow_stack

In [12]:
x = []
y = []
_, tX1, tY1 = SWP1.get(testSeries)
x.append(tX1[0])
y.append(tY1[0])

_, tX1, tY1 = SWP2.get(testSeries)
x.append(tX1[0])
y.append(tY1[0])
_, tX1, tY1 = SWP3.get(testSeries)
x.append(tX1[0])
y.append(tY1[0])
_, tX1, tY1 = SWP4.get(testSeries)
x.append(tX1[0])
y.append(tY1[0])
_, tX1, tY1 = SWP5.get(testSeries)
x.append(tX1[0])
y.append(tY1[0])

In [22]:
predictions = []
for i in range(len(workflow_stack)):
    model = workflow_stack[i][0]
    print(model)
    y = model.__test__([x[i]])
    print(y)
    # predictions.append((x[i]))

[[4.973754]]
[[4.973754]]
[[4.973754]]
[[4.973754]]
[[4.973754]]


In [23]:
workflow_stack[0][0] == workflow_stack[1][0]

True